In [ ]:
# module used
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import linear_model
from sklearn import model_selection
from sklearn.svm import SVC,SVR
from sklearn.neural_network import MLPClassifier, MLPRegressor
import sklearn.metrics as metrics
from sklearn.ensemble import StackingRegressor, StackingClassifier
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import multiprocessing as mp
from scipy import linalg, special
import pandas as pd
import statsmodels.formula.api as smf
import statistics as stat
# import seaborn as sns
import multiprocess
import time
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from scipy.linalg import toeplitz

# funcitons

In [ ]:
def simulation_base(p,n,rho,beta_gn,sz,sz0):
    # complex relationship with fixed true model
    from itertools import combinations
    p0 = 20; prest = p-p0
    interind = np.array([np.array(list(x)) for x in combinations(range(2, len(beta_gn), len(beta_gn)//5), 2)])

    z00 = np.random.binomial(n=1, p=0.35, size=n)

    # correlated continuous covariates
    sig = toeplitz([rho ** i for i in range(p0-1)])
    z0 = np.random.multivariate_normal(mean=np.zeros(p0-1), cov=sig, size=n)
    Z0 = np.concatenate([z00[:, None], z0], axis=1)
    interterm = np.apply_along_axis(lambda x: Z0[:, x[0]] * Z0[:, x[1]], axis=1, arr=interind)
    costerm = np.apply_along_axis(lambda x: np.cos(Z0[:,x]),axis = 0, arr = np.unique(interind))
    puse0 = 1/(1+(np.exp(-0.2*np.dot(Z0, beta_gn)- 0.1 * interterm.sum(axis=0))))
    t0 = np.random.binomial(n=1, p=puse0, size=n)

    y0_1 = np.random.normal(loc=sz0 + np.dot(Z0, beta_gn)+interterm.sum(axis=0), scale=1)
    y0_0 = np.random.normal(loc=np.dot(Z0, beta_gn)+interterm.sum(axis=0)+costerm.sum(axis = 1), scale=1)
    y0 = np.where(t0==1, y0_1, y0_0)

    Zaux0 = np.random.normal(0,1,size = (n,prest))

    dat0 = pd.DataFrame(np.concatenate([Z0, Zaux0, t0[:, None], y0[:, None]], axis=1), columns=["z00"] + [f"z{i}0" for i in range(1, p)] + ["t0", "y0"])

    # time point 2
    z01 = z00 + np.random.uniform(size=n)

    z1_1 = z0 + 0.1*y0_1[:, np.newaxis]
    z1_0 = z0 + 0.1*y0_0[:, np.newaxis]
    z1 = np.concatenate([z1_1[t0==1, :], z1_0[t0==0, :]], axis=0)

    Z1_1 = np.concatenate([z01[:, None], z1_1], axis=1)
    Z1_0 = np.concatenate([z01[:, None], z1_0], axis=1)
    Z1 = np.concatenate([z01[:, None], z1], axis=1)

    interterm1 = np.apply_along_axis(lambda x: Z1[:, x[0]] * Z1[:, x[1]], axis=1, arr=interind)
    costerm1 = np.apply_along_axis(lambda x:np.cos(Z1[:,x]),axis = 0, arr = np.unique(interind))
    puse1 = 1/(1+np.exp(-0.1*np.dot(Z1, beta_gn)+0.5-0.05*interterm1[range(5,interterm1.shape[0]),:].sum(axis = 0)))
    t1 = np.random.binomial(n=1, p=puse1, size=n)

    y1_11 = np.random.normal(loc=sz + np.dot(Z1_1, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0)+0.5*costerm1.sum(axis = 1), scale=1)
    y1_00 = np.random.normal(loc=np.dot(Z1_0, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0), scale=1)
    y1_10 = np.random.normal(loc=np.dot(Z1_1, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0)+0.5*costerm1.sum(axis = 1), scale=1)
    y1_01 = np.random.normal(loc=sz + np.dot(Z1_0, np.concatenate([np.array([0]), beta_gn[1:]]))+0.5*interterm1.sum(axis = 0)+0.5*costerm1.sum(axis = 1), scale=1)
    y1 = np.where(t0==1, np.where(t1==1, y1_11, y1_10), np.where(t1==0, y1_00, y1_01))

    Zaux1 = np.random.normal(0,1,size = (n,prest))

    dat1 = pd.DataFrame(np.concatenate([Z1, Zaux1, t1[:, None], y1[:, None]], axis=1), columns=["z01"] + [f"z{i}1" for i in range(1, p)] + ["t1", "y1"])
    
    full_data = pd.concat([dat0, dat1], axis=1)
    return full_data, pd.DataFrame({'y_11': y1_11, 'y_00': y1_00, 'p0': puse0, 'p1':puse1})



In [ ]:
# PS function
def ps_train_ensemble(data,Train,Test,features,outcomes,
                      include_lg=False, include_el=True, include_xgb=True, include_mlp=False,
                      include_l1=False, include_svm=False, include_nb=True,include_rf=False):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    np.random.seed(233)
    models = {}
    # base learner 1
    if include_lg:
        lg = linear_model.LogisticRegression()
        lg.fit(data[features],data[outcomes])
        models['logit'] = lg

    # base learner 2
    if include_el:
        el_class = linear_model.SGDClassifier(penalty='elasticnet',loss='log')
        param_grid_el = {'alpha': [0.0001, 0.001, 0.01, 0.1]}
        gs_el = GridSearchCV(estimator=el_class,param_grid=param_grid_el,cv=5,scoring='neg_log_loss')
        gs_el.fit(Train[features],Train[outcomes])
        el_class.set_params(**gs_el.best_params_)
        el_class.fit(Test[features],Test[outcomes])
        models['enet'] = el_class

    # base learner 3
    if include_xgb:
        param_grid_G = {
        'learning_rate': [0.01, 0.1, 1],
         'max_depth': [55,65,80],
        # 'n_estimators': [100,200,300],
        # 'alpha':[0.05,0.1,0.18],
        # 'gamma': [1,1.5,1.8]
        }
        xgb_cls = xgb.XGBClassifier(objective='binary:logistic')
        gs_xgb = GridSearchCV(
            estimator=xgb_cls,
            param_grid=param_grid_G,
            cv=5,
            scoring='neg_log_loss'
        )
        gs_xgb.fit(Train[features], Train[outcomes])
        xgb_cls.set_params(**gs_xgb.best_params_)
        xgb_cls.fit(Test[features], Test[outcomes])
        models['xgb'] = xgb_cls

    # base learner 4
    if include_mlp:
        mlp_cls = MLPClassifier()
        param_grid_mlpc = {
            'hidden_layer_sizes': [(50,), (100,), (50,50), (100,100)],
        }
        gs_mlp = GridSearchCV(estimator=mlp_cls,param_grid=param_grid_mlpc,cv=5,scoring="neg_log_loss")
        gs_mlp.fit(Train[features],Train[outcomes])
        mlp_cls.set_params(**gs_mlp.best_params_)
        mlp_cls.fit(Test[features],Test[outcomes])
        models['mlp'] = mlp_cls

    if include_l1:
        l1_cls = linear_model.LogisticRegression(penalty='l1',solver='saga')
        param_grid_l1 = {
        'C':[0.1,1,5],
        #'max_features': [None, 'sqrt', 'log2']
        }
        gs_l1 = GridSearchCV(estimator=l1_cls,param_grid=param_grid_l1,cv=5,scoring="neg_log_loss")
        gs_l1.fit(Train[features],Train[outcomes])
        l1_cls.set_params(**gs_l1.best_params_)
        l1_cls.fit(Test[features],Test[outcomes])
        models['gbr'] = l1_cls

    if include_svm:
        svm_c = SVC(kernel='linear',probability=True)
        gs_svm = GridSearchCV(estimator=svm_c,param_grid={'C': [0.1,1,10,100]},cv=5,scoring='neg_log_loss')
        gs_svm.fit(Train[features],Train[outcomes])
        svm_c.set_params(**gs_svm.best_params_)
        svm_c.fit(Test[features],Test[outcomes])
        models['svm'] = svm_c
        
    if include_nb:
        nb_c = GaussianNB()
        param_grid_nb = {
            'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
            }
        gs_nb = GridSearchCV(nb_c, param_grid=param_grid_nb, cv=5)
        gs_nb.fit(Train[features],Train[outcomes])
        nb_c.set_params(**gs_nb.best_params_)
        nb_c.fit(Test[features],Test[outcomes])
        models['nb'] = nb_c

    if include_rf:
        rf_c = RandomForestClassifier(random_state = 42)
        param_grid_rf = {
            'n_estimators':[100,200],
            'max_depth':[40,50],
            'min_samples_split':[2,5],
            }
        gs_rf = GridSearchCV(rf_c, param_grid=param_grid_rf, cv=5, scoring='neg_log_loss')
        gs_rf.fit(Train[features],Train[outcomes])
        rf_c.set_params(**gs_rf.best_params_)
        rf_c.fit(Test[features],Test[outcomes])
        models['rf'] = rf_c

    # meta learner
    Q1fit = []
    if include_lg:
        Q1fit.append(lg.predict_proba(data[features])[:,1].ravel())
    if include_el:
        Q1fit.append(el_class.predict_proba(data[features])[:,1].ravel())
    if include_xgb:
        Q1fit.append(xgb_cls.predict_proba(data[features])[:,1].ravel())
    if include_mlp:
        Q1fit.append(mlp_cls.predict_proba(data[features])[:,1].ravel())
    if include_l1:
        Q1fit.append(l1_cls.predict_proba(data[features])[:,1].ravel())
    if include_svm:
        Q1fit.append(svm_c.predict_proba(data[features])[:,1].ravel())
    if include_nb:
        Q1fit.append(nb_c.predict_proba(data[features])[:,1].ravel())
    if include_rf:
        Q1fit.append(rf_c.predict_proba(data[features])[:,1].ravel())

    cls_stack = linear_model.LogisticRegression()
    cls_stack.fit(np.transpose(Q1fit),data[outcomes])

    return models, cls_stack

def ps_function(data,features,outcomes, lg=False, el=True, xgb=True, mlp=False,
                l1=False, svm=False, nb=True,rf=False):
    import warnings
    warnings.filterwarnings('ignore')
    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    model_tr,stack_tr = ps_train_ensemble(data,Train,Test,features,outcomes, include_lg=lg, include_el=el, include_xgb=xgb, include_mlp=mlp,
                                          include_l1=l1, include_svm=svm, include_nb=nb,include_rf=rf)
    model_ts,stack_ts = ps_train_ensemble(data,Test,Train,features,outcomes, include_lg=lg, include_el=el, include_xgb=xgb, include_mlp=mlp,
                                          include_l1=l1, include_svm=svm, include_nb=nb,include_rf=rf)
    return model_tr,model_ts,stack_tr,stack_ts

def ps_predict(preddat,features,model_tr,model_ts,stack_tr,stack_ts):
    y_pred_tr = {}
    for name, model in model_tr.items():
        y_pred_tr[name] = model.predict_proba(preddat[features])[:,1].ravel()
    Q_base_tr = np.transpose(np.array(list(y_pred_tr.values())))
    eta_tr = stack_tr.predict_proba(Q_base_tr)[:,1]

    y_pred_ts = {}
    for name, model in model_ts.items():
        y_pred_ts[name] = model.predict_proba(preddat[features])[:,1].ravel()
    Q_base_ts = np.transpose(np.array(list(y_pred_ts.values())))
    eta_ts = stack_ts.predict_proba(Q_base_ts)[:,1]

    return (eta_tr+eta_ts)/2

In [ ]:
# Gcomputation function
def gcompu_train_ensemble(data,Train,Test,features,outcomes,
                          include_lm = True, include_rg = False, 
                          include_xgb = True, include_mlp = False,
                          include_gbr = True):
    import warnings
    warnings.filterwarnings('ignore')
    # Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    np.random.seed(233)
    models = {}
    # base learner 1
    if include_lm:
        lm_1 = linear_model.LinearRegression()
        lm_1.fit(data[features],data[outcomes])
        models['lm'] = lm_1

    # base learner 2
    if include_rg:
        rg_lm = linear_model.Ridge()
        param_grid_rg = {'alpha': [0.1, 1, 10]}
        gs_rg = GridSearchCV(estimator=rg_lm,param_grid=param_grid_rg,cv=5,scoring="neg_mean_absolute_error")
        gs_rg.fit(Train[features],Train[outcomes])
        rg_lm.set_params(**gs_rg.best_params_)
        rg_lm.fit(Test[features],Test[outcomes])
        models['rg'] = rg_lm

    # base learner 3
    if include_xgb:
        param_grid_G = {
        'learning_rate': [0.01, 0.1, 1],
        'max_depth': [15,20,25,35],
        # 'n_estimators': [100, 150,200],
        #'alpha':[0.05,0.1,0.18],
        #'gamma': [1,1.5,1.8]
        }
        xgb_lm1_tr = xgb.XGBRegressor()
        gs_gcomp1_tr = GridSearchCV(
            estimator=xgb_lm1_tr,
            param_grid=param_grid_G,
            cv=5,
            scoring='neg_mean_absolute_error'
        )
        gs_gcomp1_tr.fit(Train[features], Train[outcomes])
        xgb_lm1_tr.set_params(**gs_gcomp1_tr.best_params_)
        xgb_lm1_tr.fit(Test[features], Test[outcomes])
        models['xgb'] = xgb_lm1_tr

    # base learner 4
    if include_mlp:
        mlp_lm = MLPRegressor()
        param_grid_mlpr = {
            'hidden_layer_sizes': [ (100,), (150,), (100, 100), (150,150)],
            'alpha': [0.0001, 0.001, 0.01, 0.1],
        }
        gs_mlp_lm = GridSearchCV(estimator=mlp_lm,param_grid=param_grid_mlpr,cv=5,scoring="neg_mean_absolute_error")
        gs_mlp_lm.fit(Train[features],Train[outcomes])
        mlp_lm.set_params(**gs_mlp_lm.best_params_)
        mlp_lm.fit(Test[features],Test[outcomes])
        models['mlp'] = mlp_lm

    if include_gbr:
        gbr_lm = GradientBoostingRegressor()
        param_grid_gbr = {
        # 'n_estimators': [100, 200, 300],
        'learning_rate': [0.05, 0.1, 0.2],
        'max_depth': [15,20,25,35],
        #'min_samples_split': [2, 3, 4],
        #'min_samples_leaf': [1, 2, 3],
        #'max_features': [None, 'sqrt', 'log2']
        }
        gs_gbr_lm = GridSearchCV(estimator=gbr_lm,param_grid=param_grid_gbr,cv=5,scoring="neg_mean_absolute_error")
        gs_gbr_lm.fit(Train[features],Train[outcomes])
        gbr_lm.set_params(**gs_gbr_lm.best_params_)
        gbr_lm.fit(Test[features],Test[outcomes])
        models['gbr'] = gbr_lm

    # meta learner
    Q1fit = []
    if include_lm:
        Q1fit.append(lm_1.predict(data[features]).ravel())
    if include_rg:
        Q1fit.append(rg_lm.predict(data[features]).ravel())
    if include_xgb:
        Q1fit.append(xgb_lm1_tr.predict(data[features]).ravel())
    if include_mlp:
        Q1fit.append(mlp_lm.predict(data[features]).ravel())
    if include_gbr:
        Q1fit.append(gbr_lm.predict(data[features]).ravel())

    lm_stack = linear_model.LinearRegression()
    lm_stack.fit(np.transpose(Q1fit),data[outcomes])

    return models, lm_stack

def gcompu_function(data,features,outcomes,lm = True, rg = False, 
                    xgb = True, mlp = False,gbr = True):
    import warnings
    warnings.filterwarnings('ignore')
    Train,Test = train_test_split(data,test_size=0.4,random_state=42)
    model_tr,stack_tr = gcompu_train_ensemble(data,Train,Test,features,outcomes,
                                              include_lm = lm, include_rg = rg, 
                                              include_xgb = xgb, include_mlp = mlp,include_gbr = gbr)
    model_ts,stack_ts = gcompu_train_ensemble(data,Test,Train,features,outcomes,
                                              include_lm = lm, include_rg = rg, 
                                              include_xgb = xgb, include_mlp = mlp,include_gbr = gbr)
    return model_tr,model_ts,stack_tr,stack_ts


def gcompu_predict(preddat,features,model_tr,model_ts,stack_tr,stack_ts):
    y_pred_tr = {}
    for name, model in model_tr.items():
        y_pred_tr[name] = model.predict(preddat[features]).ravel()
    Q_base_tr = np.transpose(np.array(list(y_pred_tr.values())))
    eta_tr = stack_tr.predict(Q_base_tr)
    y_pred_ts = {}
    for name, model in model_ts.items():
        y_pred_ts[name] = model.predict(preddat[features]).ravel()
    Q_base_ts = np.transpose(np.array(list(y_pred_ts.values())))
    eta_ts = stack_ts.predict(Q_base_ts)

    return (eta_tr+eta_ts)/2

# Simulation Demo

In [ ]:
# coef
np.random.seed(123)
p0 = 20; 
beta_gn = np.zeros(p0)
selected_indices = np.random.choice(np.arange(p0), size=p0-10, replace=False)
beta_gn[selected_indices] = np.random.normal(size=p0-10)

In [ ]:
# funciton
def simulation_loop(b,n = 1000,p = 50,beta_gn = beta_gn):
    import warnings
    warnings.filterwarnings('ignore')
    print(p)
    np.random.seed(123+b)
    testdat, paramdat = simulation_base(p,n,0.5,beta_gn,0,0)
    phi_data = (np.mean(paramdat['y_11']-paramdat['y_00']))

    # estimate pt1 ps
    ps0outcome = ['t0']
    ps0feature = ['z' + str(x) +'0' for x in range(0,p)]
    ps1outcome = ['t1']
    ps1feature = np.concatenate((['z' + str(x) +'0' for x in range(0,p)],
                                    ['z' + str(x) +'1' for x in range(0,p)],['t0','y0']))
    #Train,Test = train_test_split(testdat,test_size = 0.4,random_state=42)

    baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts = ps_function(testdat,ps0feature,ps0outcome)
    enps0 = ps_predict(testdat,ps0feature,baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts)

    baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts = ps_function(testdat,ps1feature,ps1outcome,l1=True,mlp=True)
    enps1 = ps_predict(testdat,ps1feature,baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts)



    ## outcome regression ensemble
    iter1feature = ['z{}{}'.format(i, j) for i in range(p) for j in range(2)] + ['y0', 't0', 't1']
    iter1outcome = ['y1']
    iter0feature = ['t0']+['z' + str(x) +'0' for x in range(0,p)]

    preddat11 = testdat.copy(); preddat11['t0']=1;preddat11['t1']=1
    preddat00 = testdat.copy(); preddat00['t0']=0;preddat00['t1']=0
    
    basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts = gcompu_function(testdat,iter1feature,iter1outcome)

    eta2 = gcompu_predict(testdat,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
    

    # G-computation
     ## G-comp with LR
    lm1 = linear_model.LinearRegression()
    lm1.fit(testdat[iter1feature],testdat[iter1outcome])
    Q11 = lm1.predict(preddat11[iter1feature])
    Q00 = lm1.predict(preddat00[iter1feature])

    lm0_1 = linear_model.LinearRegression()
    lm0_1.fit(testdat[iter0feature],Q11)
    lm0_0 = linear_model.LinearRegression()
    lm0_0.fit(testdat[iter0feature],Q00)
    phi_gcomp_lm = (np.mean(lm0_1.predict(preddat11[iter0feature])-lm0_0.predict(preddat00[iter0feature])))

    ## DR
    
    #eta2 = lm_stack.predict(Q1fit)

    preddat1 = testdat.copy(); preddat1['t1']=1
    preddat0 = testdat.copy(); preddat0['t1']=0


    eta20 = gcompu_predict(preddat0,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
    basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts = gcompu_function(pd.concat([testdat, pd.DataFrame(eta20.reshape(n,), columns=['eta20'])], axis=1),
                                                    iter0feature,'eta20')

    eta21 = gcompu_predict(preddat1,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
    basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts = gcompu_function(pd.concat([testdat, pd.DataFrame(eta21.reshape(n,), columns=['eta21'])], axis=1),
                                                    iter0feature,'eta21')


    Q_00= gcompu_predict(preddat00,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    Q_11= gcompu_predict(preddat11,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)

    phi_gcomp = (np.mean(Q_11-Q_00))
            

    eta10 = gcompu_predict(preddat0,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    eta11 = gcompu_predict(preddat1,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)

    eta0_00 = Q_00

    preddat10 = testdat.copy(); preddat10['t0']=1;preddat10['t1']=0
    eta0_10 = gcompu_predict(preddat10,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)

    eta0_11 = Q_11

    preddat01 = testdat.copy(); preddat01['t0']=0;preddat01['t1']=1
    eta0_01 = gcompu_predict(preddat01,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)


    ## linear equation for DR, subject to change based on MSM/other outcome equation of interest
    t0 = testdat['t0']; t1= testdat['t1']; y1 = testdat['y1']; y0 = testdat['y0']
    f01 = ((t0/enps0+(1-t0)/(1-enps0))*(t1/enps1+(1-t1)/(1-enps1))*(y1-eta2.ravel()))
    f02 = 1/((enps0**t0)*(1-enps0)**(1-t0))*(eta20.ravel()-eta10.ravel())+1/((enps0**t0)*(1-enps0)**(1-t0))*(eta21.ravel()-eta11.ravel())
    f03 = (eta0_00+eta0_11+eta0_10.ravel()+eta0_01.ravel())
    c0 = np.mean(f01+f02+f03)
    b0 = [4,2,2]

    f11 = (t0/enps0+(1-t0)/(1-enps0))*(t1/enps1+(1-t1)/(1-enps1))*(y1-eta2.ravel())*t0
    f12 = t0/((enps0**t0)*(1-enps0)**(1-t0))*(eta20.ravel()-eta10.ravel())+t0/((enps0**t0)*(1-enps0)**(1-t0))*(eta21.ravel()-eta11.ravel())
    f13 = (eta0_11+eta0_10.ravel())
    c1 = np.mean(f11+f12+f13)
    b1 = [2,2,1]

    f21 = (t0/enps0+(1-t0)/(1-enps0))*(t1/enps1+(1-t1)/(1-enps1))*(y1-eta2.ravel())*t1
    f22 = 1/((enps0**t0)*(1-enps0)**(1-t0))*(eta21.ravel()-eta11.ravel())
    f23 = (eta0_11+eta0_01.ravel())
    c2 = np.mean(f21+f22+f23)
    b2 = [2,1,2]

    coef = np.linalg.solve(np.array([b0,b1,b2]),np.array([c0,c1,c2]))

    phi_dr = (coef[1]+coef[2])


    ## MSM with original ps
    #puse0 = paramdat['p0']; puse1 = paramdat['p1']
    IPW = (t0/enps0+(1-t0)/(1-enps0))*(t1/enps1+(1-t1)/(1-enps1))
    msm_lm = linear_model.LinearRegression()
    msm_lm.fit(testdat[['t0','t1']],testdat['y1'],sample_weight=IPW)
    phi_msm_lm = (np.sum(msm_lm.coef_))

    ## bootstrap 
    dr_boot = []; dr_boot_wo = []; # wo: without re-estimation of nuisance parameters
    ps_boot = []; msm_boot = []; msm_b1boot = []; msm_b2boot = []
    gcomp_boot = []; gcomplm_boot = []
    for i in range(500):
        sampleid = np.random.choice(range(1, n+1), size=n, replace=True)
        dat_boot = testdat.iloc[sampleid-1, :]

        enps0_btwo = ps_predict(dat_boot,ps0feature,baseps0_tr,baseps0_ts,cls_stack0_tr,cls_stack0_ts)

        enps1_btwo = ps_predict(dat_boot,ps1feature,baseps1_tr,baseps1_ts,cls_stack1_tr,cls_stack1_ts)

        bootdat11 = dat_boot.copy(); bootdat11['t0']=1;bootdat11['t1']=1
        bootdat00 = dat_boot.copy(); bootdat00['t0']=0;bootdat00['t1']=0

        bootdat1 = dat_boot.copy(); bootdat1['t1']=1
        bootdat0 = dat_boot.copy(); bootdat0['t1']=0

        bootdat10 = dat_boot.copy(); bootdat10['t0']=1;bootdat10['t1']=0
        bootdat01 = dat_boot.copy(); bootdat01['t0']=0;bootdat01['t1']=1


        ## G-comp with LR
        lm1boot = linear_model.LinearRegression()
        lm1boot.fit(dat_boot[iter1feature],dat_boot[iter1outcome])
        Q11boot = lm1boot.predict(bootdat11[iter1feature])
        Q00boot = lm1boot.predict(bootdat00[iter1feature])

        lm0_1boot = linear_model.LinearRegression()
        lm0_1boot.fit(dat_boot[iter0feature],Q11boot)
        lm0_0boot = linear_model.LinearRegression()
        lm0_0boot.fit(dat_boot[iter0feature],Q00boot)
        gcomplm_boot.append(np.mean(lm0_1boot.predict(bootdat11[iter0feature])-lm0_0boot.predict(bootdat00[iter0feature])))


        # PS

        ## MSM
        IPWboot = (dat_boot['t0'].values.ravel()/enps0_btwo+(1-dat_boot['t0'].values.ravel())/(1-enps0_btwo))*(dat_boot['t1'].values.ravel()/enps1_btwo+(1-dat_boot['t1'].values.ravel())/(1-enps1_btwo))
        msmboot_lm = linear_model.LinearRegression()
        msmboot_lm.fit(dat_boot[['t0','t1']],dat_boot['y1'],sample_weight=IPWboot)
        msm_boot.append((np.sum(msmboot_lm.coef_))); msm_b1boot.append(msmboot_lm.coef_[0]); msm_b2boot.append(msmboot_lm.coef_[1])

        eta2_btwo = gcompu_predict(dat_boot,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)

        eta20_btwo = gcompu_predict(bootdat0,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
        eta21_btwo = gcompu_predict(bootdat1,iter1feature,basemodel1_tr,basemodel1_ts,lm_stack1_tr,lm_stack1_ts)
    
        eta10_btwo = gcompu_predict(bootdat0,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)
    
        eta11_btwo = gcompu_predict(bootdat1,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)

        eta0_00_btwo = gcompu_predict(bootdat00,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)
    
        eta0_11_btwo = gcompu_predict(bootdat11,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)

        eta0_10_btwo = gcompu_predict(bootdat10,iter0feature,basemodel20_tr,basemodel20_ts,lm_stack20_tr,lm_stack20_ts)
    
        eta0_01_btwo = gcompu_predict(bootdat01,iter0feature,basemodel21_tr,basemodel21_ts,lm_stack21_tr,lm_stack21_ts)


        # DR step
        boott0 = dat_boot['t0']; boott1= dat_boot['t1']; booty1 = dat_boot['y1']; #booty0 = dat_boot['y0']
        f01_btwo = ((boott0/enps0_btwo+(1-boott0)/(1-enps0_btwo))*(boott1/enps1_btwo+(1-boott1)/(1-enps1_btwo))*(booty1-eta2_btwo.ravel()))
        f02_btwo = 1/((enps0_btwo**boott0)*(1-enps0_btwo)**(1-boott0))*(eta20_btwo.ravel()-eta10_btwo.ravel())+1/((enps0_btwo**boott0)*(1-enps0_btwo)**(1-boott0))*(eta21_btwo.ravel()-eta11_btwo.ravel())
        f03_btwo = (eta0_00_btwo+eta0_11_btwo+eta0_10_btwo.ravel()+eta0_01_btwo.ravel())
        c0_btwo = np.mean(f01_btwo+f02_btwo+f03_btwo)
        b0_btwo = [4,2,2]

        f11_btwo = (boott0/enps0_btwo+(1-boott0)/(1-enps0_btwo))*(boott1/enps1_btwo+(1-boott1)/(1-enps1_btwo))*(booty1-eta2_btwo.ravel())*boott0
        f12_btwo = boott0/((enps0_btwo**boott0)*(1-enps0_btwo)**(1-boott0))*(eta20_btwo.ravel()-eta10_btwo.ravel())+boott0/((enps0_btwo**boott0)*(1-enps0_btwo)**(1-boott0))*(eta21_btwo.ravel()-eta11_btwo.ravel())
        f13_btwo = (eta0_11_btwo+eta0_10_btwo.ravel())
        c1_btwo = np.mean(f11_btwo+f12_btwo+f13_btwo)
        b1_btwo = [2,2,1]

        f21_btwo = (boott0/enps0_btwo+(1-boott0)/(1-enps0_btwo))*(boott1/enps1_btwo+(1-boott1)/(1-enps1_btwo))*(booty1-eta2_btwo.ravel())*boott1
        f22_btwo = 1/((enps0_btwo**boott0)*(1-enps0_btwo)**(1-boott0))*(eta21_btwo.ravel()-eta11_btwo.ravel())
        f23_btwo = (eta0_11_btwo+eta0_01_btwo.ravel())
        c2_btwo = np.mean(f21_btwo+f22_btwo+f23_btwo)
        b2_btwo = [2,1,2]

        coef_btwo = np.linalg.solve(np.array([b0_btwo,b1_btwo,b2_btwo]),np.array([c0_btwo,c1_btwo,c2_btwo]))

        dr_boot_wo.append(coef_btwo[1]+coef_btwo[2])


    return  phi_data, phi_gcomp, phi_dr,phi_gcomp_lm, phi_msm_lm, dr_boot_wo, msm_boot,msm_b1boot,msm_b2boot,gcomp_boot,gcomplm_boot

In [ ]:
# simulation with p = 100, effect size = 5
from functools import partial
start_time = time.time()
with multiprocess.Pool(processes=20) as pool:
    # phi_data_100bs, phi_gcomp_100bs, phi_dr_100bs, phi_gcomp_lm_100bs, phi_msm_lm_100bs, dr_boot_100bs, dr_btwo_100bs = pool.map(simulation_loop,range(10))
    results2 = pool.map(partial(simulation_loop,n=500,p = 100),range(80))
    pool.close(); pool.join()

end_time = time.time()